In [ ]:
!pip install open3d pyntcloud

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 408.6 MB 28 kB/s 
     |████████████████████████████████| 346 kB 51.9 MB/s 
     |████████████████████████████████| 1.1 MB 51.6 MB/s 
     |████████████████████████████████| 596 kB 55.4 MB/s 
     |████████████████████████████████| 3.1 MB 50.6 MB/s 
     |████████████████████████████████| 8.8 MB 43.5 MB/s 
     |████████████████████████████████| 428 kB 41.8 MB/s 
     |████████████████████████████████| 344 kB 60.9 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
     |████████████████████████████████| 1.2 MB 39.1 MB/s 
     |████████████████████████████████| 80 kB 4.1 MB/s 
     |████████████████████████████████| 54 kB 2.4 MB/s 
     |████████████████████████████████| 561 kB 32.4 MB/s 
     |████████████████████████████████| 132 kB 6.5 MB/s 
     |████████████████████████████████| 133 kB 35.2 MB/s 
     |████████████████████████████████

In [ ]:
import open3d as o3d
import os
path = "/content/Solider.ply.bin.decoded.ply"
import numpy as np
from pyntcloud import PyntCloud

In [ ]:
# Reading the point cloud using PCD
pcd_original = o3d.io.read_point_cloud(path)
print(pcd_original)

[Open3D WARNING] Read PLY failed: unable to read file: /content/Solider.ply.bin.decoded.ply
PointCloud with 1089091 points.


In [ ]:
pcd_xyz = np.asarray(pcd_original.points)
pcd_rgb = np.asarray(pcd_original.colors)

In [ ]:
full_pc = np.concatenate((pcd_xyz,pcd_rgb),axis=1)

In [ ]:
original_indexes = range(0,len(full_pc))
original_indexes

range(0, 1089091)

In [ ]:
import time
original_indexes= list(original_indexes)
patch_pcs =[]
iteration=0
start_time = time.time()
while original_indexes:
    dists = ((full_pc[original_indexes[0]][0:3] - full_pc[original_indexes,0:3])**2).sum(-1) # Calculating distances relative to all the remaining points
    patch = sorted(enumerate(dists), key=lambda i: i[1]) # (index , distance) sorted by distance
    if iteration%5==0:
        print("Reached iteration : ",iteration)
        print("Points remaining : ",len(patch))
        print("--- %s seconds ---" % (time.time() - start_time))
    iteration+=1 
    if(len(patch)>=2048):
        patch =patch[0:2048] # Taking the closest 2048 points
    else:
        patch =patch[0:] # In case we reach the last block and it has less then 2048 points
    patch_pc=[] # Placeholder for a single patch with exact points
    to_be_removed_inds = []
    for p in patch: # for each index in the patch
        patch_pc.append(full_pc[p[0]].tolist()) # add the real points to patch_pc 
        original_indexes.remove(p[0]) # remove the index from the original indexes
        to_be_removed_inds.append(p[0])
    full_pc = full_pc[[x for x in range(len(full_pc)) if x not in to_be_removed_inds]]
    original_indexes=list(range(0,len(original_indexes)))
    patch_pcs.append(patch_pc)

Reached iteration :  0
Points remaining :  1089091
--- 1.033198595046997 seconds ---
Reached iteration :  5
Points remaining :  1078851
--- 173.8842146396637 seconds ---
Reached iteration :  10
Points remaining :  1068611
--- 350.258332490921 seconds ---
Reached iteration :  15
Points remaining :  1058371
--- 522.9354429244995 seconds ---
Reached iteration :  20
Points remaining :  1048131
--- 697.8018498420715 seconds ---
Reached iteration :  25
Points remaining :  1037891
--- 864.1631572246552 seconds ---
Reached iteration :  30
Points remaining :  1027651
--- 1032.3696548938751 seconds ---
Reached iteration :  35
Points remaining :  1017411
--- 1199.5375401973724 seconds ---
Reached iteration :  40
Points remaining :  1007171
--- 1364.2339172363281 seconds ---
Reached iteration :  45
Points remaining :  996931
--- 1528.3800721168518 seconds ---
Reached iteration :  50
Points remaining :  986691
--- 1688.2720890045166 seconds ---
Reached iteration :  55
Points remaining :  976451
---

In [8]:
patch_pcs[0]

[[125.0, 16.0, 36.0, 0.0, 0.27450980392156865, 0.2627450980392157],
 [125.0,
  17.0,
  36.0,
  0.2901960784313726,
  0.2823529411764706,
  0.27058823529411763],
 [125.0,
  16.0,
  37.0,
  0.3333333333333333,
  0.2980392156862745,
  0.28627450980392155],
 [126.0,
  16.0,
  36.0,
  0.2823529411764706,
  0.27058823529411763,
  0.25882352941176473],
 [124.0,
  17.0,
  36.0,
  0.2901960784313726,
  0.29411764705882354,
  0.2823529411764706],
 [125.0,
  17.0,
  35.0,
  0.3254901960784314,
  0.26666666666666666,
  0.2549019607843137],
 [125.0,
  15.0,
  37.0,
  0.35294117647058826,
  0.29411764705882354,
  0.2823529411764706],
 [125.0,
  17.0,
  37.0,
  0.3607843137254902,
  0.30196078431372547,
  0.2901960784313726],
 [126.0,
  15.0,
  36.0,
  0.2784313725490196,
  0.26666666666666666,
  0.2549019607843137],
 [126.0,
  16.0,
  35.0,
  0.2901960784313726,
  0.25882352941176473,
  0.24705882352941178],
 [124.0,
  17.0,
  37.0,
  0.35294117647058826,
  0.3137254901960784,
  0.30196078431372547]

In [9]:
numberp = 2048 - len(patch_pcs[-1])
numberp

445

In [10]:
for i in range(0,numberp):
  patch_pcs[-1].append(patch_pcs[-1][-1])

In [11]:
len(patch_pcs[-1])

2048

In [12]:
import numpy as np
np_patches = np.asarray(patch_pcs)

In [13]:
pcdexp = o3d.geometry.PointCloud()
pcdexp.points = o3d.utility.Vector3dVector(np_patches[1][:,0:3])
pcdexp.colors = o3d.utility.Vector3dVector(np_patches[1][:,3:6])
o3d.io.write_point_cloud("exp_patch.ply", pcdexp)

True

In [15]:
for i in range(0,len(np_patches)):
    pc = o3d.geometry.PointCloud() #Creating the pointcloud
    pc.points= o3d.utility.Vector3dVector(np_patches[i][:,0:3]) # Setting geometry
    pc.colors= o3d.utility.Vector3dVector(np_patches[i][:,3:6]) # Setting attributes
    name= "soldier/patch_"+str(i)+".ply"
    o3d.io.write_point_cloud(name, pc)

In [17]:
!zip -r /content/solider.zip /content/soldier


  adding: content/soldier/ (stored 0%)
  adding: content/soldier/patch_71.ply (deflated 80%)
  adding: content/soldier/patch_509.ply (deflated 75%)
  adding: content/soldier/patch_236.ply (deflated 78%)
  adding: content/soldier/patch_414.ply (deflated 75%)
  adding: content/soldier/patch_400.ply (deflated 78%)
  adding: content/soldier/patch_471.ply (deflated 75%)
  adding: content/soldier/patch_324.ply (deflated 76%)
  adding: content/soldier/patch_32.ply (deflated 78%)
  adding: content/soldier/patch_259.ply (deflated 81%)
  adding: content/soldier/patch_40.ply (deflated 80%)
  adding: content/soldier/patch_355.ply (deflated 78%)
  adding: content/soldier/patch_231.ply (deflated 78%)
  adding: content/soldier/patch_429.ply (deflated 76%)
  adding: content/soldier/patch_126.ply (deflated 79%)
  adding: content/soldier/patch_458.ply (deflated 77%)
  adding: content/soldier/patch_326.ply (deflated 78%)
  adding: content/soldier/patch_86.ply (deflated 78%)
  adding: content/soldier/patc